In [1]:
import cvxpy as cvx
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.linear_model import ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation
from sklearn.cross_validation import KFold
import numpy as np
from scipy.stats import hmean
from sklearn import metrics
from sklearn.pipeline import Pipeline
from FeatureWeights import FeatureWeights
import os


import matplotlib.pyplot as plt
%matplotlib inline

Vendor:  Continuum Analytics, Inc.
Package: mkl
Message: trial mode expires in 15 days


##Load data

In [2]:
src_path = '../data/'
file_types = ('.csv')
 
file_paths = []  
for root, dirs, files in os.walk(src_path):
    file_paths.extend([os.path.join(root, f) for f in files if f.endswith(file_types)])
    
print 'number of files is', len(file_paths)

number of files is 5


In [3]:
biomarkers = ["Stool.Lysozyme",
              "Stool.Lactoferrin", 
              "Stool.SIgA",
              "X..SCFA.Acetate",
              "X..SCFA.Propionate",
              "X..SCFA.Valerate",
              "X..SCFA.Butyrate",
              "Total.SCFA",
              "Butyrate",
              "Stool.pH",
              "Neutrophil.Count",
              "Lymphocyte.Count" ,
              "Monocyte.Count",
              "Esoinophil.Count"]


biomarker = biomarkers[1]
#biomarker = "Neutrophil.Count"

In [4]:
df = pd.read_csv(file_paths[0])
fields = list(df.columns)
microbiome_indx = [i for i, field in enumerate(fields) if "__" in field]
column_names = list(df.columns)
target_indx = [i for i, column_name in enumerate(column_names) if column_name == biomarker]
df_rel = df.iloc[:,target_indx + range(microbiome_indx[0], df.shape[1])]
df_rel[[biomarker]] = df_rel[[biomarker]].apply(lambda x: pd.to_numeric(x, errors = "coerce"))
df_clean = df_rel.dropna()
y = np.array(df_clean[biomarker])
X = np.array(df_clean.iloc[:,1:])

/Users/myazdaniUCSD/anaconda/lib/python2.7/site-packages/pandas/core/frame.py:2378: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


## Compute pairwise differences

In [5]:
FW = FeatureWeights()

In [6]:
X_rel = FW.fit_transform(X, y)

In [7]:
X_rel.shape

(30, 10)

In [8]:
X_rel[0,:]

array([ 39.54453409,   0.        ,   1.20213033,  55.9488    ,
         1.47840005,   0.        ,   3.0704    ,   5.431     ,
        11.75973601,  42.2688    ])

In [10]:
scoring_metric = "mean_absolute_error"


def CV_scores(regressor, X, y, data_type, estimator_name):
    y_n = (y - np.mean(y))/np.std(y)
    scores = -1*cross_validation.cross_val_score(regressor, X, y, cv=5, scoring = scoring_metric)
    results = {'data': data_type, "model": estimator_name, "Mean score": np.mean(scores), "STD score": np.std(scores)}
    return results

In [11]:
FW = FeatureWeights()

KNN = GridSearchCV(estimator=KNeighborsRegressor(), param_grid=dict(n_neighbors=range(1,11), p=[1,2]), n_jobs = -1, 
                           scoring = scoring_metric)        


metric_KNN = Pipeline([('metric', FW), ('knn', KNN)])
CV_scores(KNN,X, y, data_type = "sample", estimator_name = "KNN")

{'Mean score': 99.089777777777769,
 'STD score': 15.527085886394026,
 'data': 'sample',
 'model': 'KNN'}

In [12]:
CV_scores(metric_KNN,X, y, data_type = "sample", estimator_name = "Metric KNN")

{'Mean score': 84.871111111111105,
 'STD score': 18.678295547717983,
 'data': 'sample',
 'model': 'Metric KNN'}